In [ ]:
from __future__ import print_function, division
%matplotlib notebook

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from IPython.core.debugger import set_trace
import itertools
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
plt.ion()

# Pose Output Format (BODY_25)
<img src="https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/keypoints_pose_25.png" width="300">

# Check cuda.is_available ?

In [ ]:
cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("cuda_available : {}, device : {}".format(cuda_available, device))

In [ ]:
part_pairs = [
              #(1,8),
              #(1,2),
              #(1,5),
              #(2,3),
              #(3,4),
              #(5,6),
              #(6,7),
              (8,9),
              (9,10),
              (10,11),
              (8,12),
              (12,13),
              (13,14),
              #(1,0),
              #(0,15),
              #(15,17),
              #(0,16),
              #(16,18),
              #(2,17),
              #(5,18),
              (14,19),
              (19,20),
              (14,21),
              (11,22),
              (22,23),
              (11,24)]

# Define Dataset & DataLoader

In [ ]:
MAXLEN = 300
FPS = 24.0

keypoints_frame = pd.read_pickle("../../preprocess/data/keypoints_dataframe.pkl")

def drop_huge_seq(keypoints_frame, save_path="../../preprocess/data/keypoints_dataframe_drop.pkl"):
    if os.path.exists(save_path):
        print('Already dropped! Return...')
        return
    
    vids = list(set(keypoints_frame.vid))

    for i in tqdm(range(len(vids)), desc='DropInputSeq '):
        slice_df = keypoints_frame.loc[keypoints_frame.vid==vids[i]].iloc[:, 2:]
        if slice_df.values.shape[0] > MAXLEN:
            keypoints_frame.iloc[slice_df.index, 2:] = np.nan * np.empty_like(slice_df.values)

    # drop Nans !
    keypoints_frame = keypoints_frame.dropna()
    keypoints_frame.to_pickle("../../preprocess/data/keypoints_dataframe_drop.pkl")

    
# drop huge seq
drop_huge_seq(keypoints_frame, save_path="../../preprocess/data/keypoints_dataframe_drop.pkl")

In [ ]:
def pid2vid(pid):
    num, test_id, trial_id = pid.split('_')
    return '_'.join([num, 'test', test_id, 'trial', trial_id]) + '.avi'
    

def vid2pid(vid):
    split = os.path.splitext(vid)[0].split('_')
    return '_'.join([split[0], split[2], split[4]])
        
    
class GAITDataset(Dataset):
    def __init__(self, keypoints_pkl_file, targets_pkl_file, phase, split_ratio, maxlen=300, fps=24.0, r_seed=3):
        keypoints_frame = pd.read_pickle(keypoints_pkl_file)
        targets_frame = pd.read_pickle(targets_pkl_file)[['Toe In / Out/L','start','end']]
        self.maxlen = maxlen
        self.fps = fps
                        
        vids = list(set(keypoints_frame.vid))

        # reindex tgt data (to filter-out valid vids)
        pids = []
        for vid in vids:
            pids.append(vid2pid(vid))

        targets_frame = targets_frame.reindex(pids, fill_value=0.0)
        
        # vids without zero values in target df
        vids = [ pid2vid(pid) for pid in targets_frame.index.values ]

        # target dataframe (minmax scaled)
        #self.target_scaler = MinMaxScaler(feature_range=(0, 1.0))
        self.target_scaler = StandardScaler()
        scaled_values = self.target_scaler.fit_transform(targets_frame.values[:,:-2])
        targets_frame.loc[:,:-2] = scaled_values
        
        self.keypoints_frame = keypoints_frame
        self.targets_frame = targets_frame
        
        import random
        random.seed(r_seed)
        random.shuffle(vids)  # shuffle vids inplace, before datasplit
        
        if phase=='train':
            self.vids = vids[:int(len(vids)*split_ratio)]
        elif phase=='test':
            self.vids = vids[-int(len(vids)*split_ratio):]

    def __len__(self):
        return len(self.vids)
    
    def __getitem__(self, idx):
        cur_keypoints_frame = self.keypoints_frame.loc[self.keypoints_frame.vid==self.vids[idx]]
        
        pid, _, test_ix, _, trial_ix, *_ = os.path.splitext(self.vids[idx])[0].split('_')
        
        target_id = '_'.join([pid, test_ix, trial_ix])
        target_data = self.targets_frame.loc[target_id].values
        
        # parse target data
        targets, (start, end) = target_data[:-2], target_data[-2:]
        
        # convert time -> frame_ix
        start_ix = int(start * FPS)
        end_ix = int(end*FPS)

        # select keypoints based on start/end time
        keypoints_seq = cur_keypoints_frame.query(f'{start_ix} < frame_cnt < {end_ix}').iloc[:, 2:].values
        
        # seq_len before padding
        seq_len = len(keypoints_seq)
        
        # zero padding
        keypoints_seq = np.pad(keypoints_seq, ((0,self.maxlen-len(keypoints_seq)),(0,0)),
                                               'constant', constant_values=0).transpose(1,0)
        
        
        sample = {'keypoints_seq': torch.tensor(keypoints_seq, dtype=torch.float32),
                  'targets': torch.tensor(targets, dtype=torch.float32),
                  'seq_len': torch.tensor(seq_len, dtype=torch.int32)}
        
        return sample

# dataset path
keypoints_pkl_file = "../../preprocess/data/keypoints_dataframe_drop.pkl"
targets_pkl_file = "../../preprocess/data/targets_dataframe.pkl"

In [ ]:
dataset = { phase : GAITDataset(keypoints_pkl_file, targets_pkl_file, phase=phase, split_ratio=split_ratio, 
                                maxlen=MAXLEN, fps=FPS) \
                for phase,split_ratio in zip(['train', 'test'], [0.8, 0.2]) }

dataloader = { phase : DataLoader(dataset[phase],
                        batch_size=8*torch.cuda.device_count(),
                        shuffle=True,
                        num_workers=16) \
                    for phase in ['train', 'test'] }

# Visualize distribution of data

In [ ]:
import seaborn as sns

df_list = []

for phase in ['train', 'test']:
    df_list.append(
        dataset[phase].targets_frame
    )

df = pd.concat(df_list)


single_columns = df.columns[:3]
pair_columns = df.columns[3:-2]

def Normal(x, mu=0.0, sigma=1.0):
    return (1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2) ))


def visualize_dist_of_factors(df, columns, fig_size=(7,7), subplot_coord=(3,1), paired=False, title=None):

    sns.set(style="white", palette="muted", color_codes=True)

    # Set up the matplotlib figure
    f, axes = plt.subplots(*subplot_coord, figsize=fig_size, sharex=False)
    if title:
        f.suptitle(title, fontsize=16, y=2.0)
    sns.despine(left=True)
    
    for i in range(len(columns)):
        ax = axes[i//2, i%2] if paired else axes[i]
        values = df[columns[i]].values
        mu, sigma = values.mean(), values.std()
        cnt, bins, _ = ax.hist(values, 60, density=True)
        
        ax.plot(bins, Normal(bins, mu, sigma))
        ax.set_title(columns[i])
        plt.tight_layout()
    plt.setp(axes, yticks=[])
    plt.tight_layout()


visualize_dist_of_factors(df, single_columns, fig_size=(7,7), subplot_coord=(3,1), paired=False, title='Single Factor Distributions')

visualize_dist_of_factors(df, pair_columns, fig_size=(9,9), subplot_coord=(7,2), paired=True, title='Paired Factor Distributions')

# Define DNN

In [ ]:
class Conv1d_with_same_padding(nn.Conv1d):
    def __init__(self, in_channels,
                       out_channels,
                       kernel_size,
                       stride=1,
                       padding=0,
                       dilation=1,
                       groups=1,
                       bias=True,
                       padding_type='same'):
        
        super(Conv1d_with_same_padding, self).__init__(in_channels,
                                     out_channels,
                                     kernel_size,
                                     stride,
                                     padding,
                                     dilation,
                                     groups,
                                     bias)
        
        self.padding_type = padding_type
    
    def forward(self, x, debug=False):
        _, _, input_length = x.size()
        if debug:
            set_trace()
        if self.padding_type == 'same':
            padding_need = round((input_length * (self.stride[0]-1) + self.kernel_size[0] - self.stride[0]) / 2)
            
        return F.conv1d(x, self.weight, self.bias, self.stride, 
                        padding_need, self.dilation, self.groups)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, C_in, C_out, pool, highway=True):
        super(ResidualBlock, self).__init__()
        self.pool = pool
        self.highway = highway
                
        stride = 1
        
        if C_in != C_out:
            C = C_out
        else:
            C = C_in = C_out
            
        if pool:
            # input dimension matchig
            self.conv_1x1_matching = Conv1d_with_same_padding(C_in, C, kernel_size=1, stride=1, padding_type='same')
            self.bn_1x1_matching = nn.BatchNorm1d(C)

            # for pooling of residual path
            stride = 2
            self.conv_1x1_pool = Conv1d_with_same_padding(C_in, C, kernel_size=1, stride=2, padding_type='same')
            self.bn_1x1_pool= nn.BatchNorm1d(C)
                
        # conv_1x1_a : reduce number of channels by factor of 4 (output_channel = C/4)
        self.conv_1x1_a = Conv1d_with_same_padding(C, int(C/4), kernel_size=1, stride=stride, padding_type='same')
        self.bn_1x1_a = nn.BatchNorm1d(int(C/4))
        
        # conv_3x3_b : more wide receptive field (output_channel = C/4)
        self.conv_3x3_b = Conv1d_with_same_padding(int(C/4), int(C/4), kernel_size=3, stride=1, padding_type='same')
        self.bn_3x3_b = nn.BatchNorm1d(int(C/4))
        
        # conv_1x1_c : recover org channel C (output_channel = C)
        self.conv_1x1_c = Conv1d_with_same_padding(int(C/4), C, kernel_size=1, stride=1, padding_type='same')
        self.bn_1x1_c = nn.BatchNorm1d(C)
        
        if highway:
            # conv_1x1_g : gating for highway network
            self.conv_1x1_g = Conv1d_with_same_padding(C, C, kernel_size=1, stride=1, padding_type='same')
        
        # output
        self.bn_1x1_out = nn.BatchNorm1d(C)
        
    
    def forward(self, x):
        '''
            x : size = (batch, C, maxlen)
        '''
        
        res = x
        
        if self.pool:
            # input dimension matching with 1x1 conv
            x = self.conv_1x1_matching(x)
            x = self.bn_1x1_matching(x)
            
            # pooling of residual path
            res = self.conv_1x1_pool(res)
            res = self.bn_1x1_pool(res)
        
        # 1x1_a (C/4)
        x = self.conv_1x1_a(x)
        x = self.bn_1x1_a(x)
        x = F.relu(x)
        
        # 3x3_b (C/4)
        x = self.conv_3x3_b(x)
        x = self.bn_3x3_b(x)
        x = F.relu(x)
        
        # 1x1_c (C)
        x = self.conv_1x1_c(x)
        x = self.bn_1x1_c(x)
        
        if self.highway:
            # gating mechanism from "highway network"
            
            # gating factors controll intensity between x and f(x)
            # gating = 1.0 (short circuit) --> output is identity (same as initial input)
            # gating = 0.0 (open circuit)--> output is f(x) (case of non-residual network)
            gating = torch.sigmoid(self.conv_1x1_g(x))
            
            # apply gating mechanism
            x = gating * res + (1.0 - gating) * F.relu(x)

            
        else:
            # normal residual ops (addition)
            x = F.relu(x) + res
            
#         x = self.bn_1x1_out(x)
#         x = F.relu(x)
        
        return x

# Get statiscal values (mean, std)

In [ ]:
df_list = []  # list of df to merge

for phase in ['train', 'test']:
    df_list.append(
        dataset[phase].targets_frame.iloc[:,:-2]
    )

df = pd.concat(df_list)  #  concat

# mean and std from data
mu = df.values.mean(0)
sigma = df.values.std(0)

In [ ]:

class View(nn.Module):
    def __init__(self, *shape):
        super(View, self).__init__()
        self.shape = shape
    def forward(self, x):
        return x.view(*self.shape)

class GAP(nn.Module):
    def __init__(self):
        super(GAP, self).__init__()
    def forward(self, x):
        '''
            x : size = (B, C, L)
        '''
        return torch.mean(x, 2)
        
        
class Net(nn.Module):
    def __init__(self, input_size, target_size, num_layers = [3,4,6], num_filters = [64,128,128]):
        super(Net, self).__init__()
        
        self.input_size = input_size
        self.target_size = target_size
        
        def res_blocks(residual_blocks, num_layers, num_filters, block_ix, pool_first_layer=True):
            block_layers = num_layers[block_ix]

            for i in range(block_layers):
                # default values
                pool = False
                block_filters = num_filters[block_ix]
                
                C_in = C_out = block_filters
                
                if pool_first_layer and i==0:
                    pool = True
                if i==0 and block_ix > 0:
                    C_in = num_filters[block_ix-1]
                    
                print(f"layer : {i}, block : {block_ix}, C_in/C_out : {C_in}/{C_out}")
                residual_blocks.append(ResidualBlock(C_in=C_in, C_out=C_out,pool=pool, highway=True))
                
        residual_blocks = []

        for i in range(len(num_layers)):
            pool_first_layer = True
            if i == 0:
                pool_first_layer = False
            res_blocks(residual_blocks, num_layers=num_layers, num_filters=num_filters, block_ix=i,
                       pool_first_layer=pool_first_layer)

        self.model = nn.Sequential(nn.Conv1d(input_size, num_filters[0], kernel_size=4, stride=2, padding=1),
                                   nn.BatchNorm1d(num_filters[0]),
                                   nn.ReLU(),
                                   nn.MaxPool1d(kernel_size=(3,), stride=2,),
                                   nn.Conv1d(num_filters[0], num_filters[0], kernel_size=4, stride=2, padding=1),
                                   nn.BatchNorm1d(num_filters[0]),
                                   nn.ReLU(),
                                   nn.MaxPool1d(kernel_size=(3,), stride=2,),
                                   nn.Conv1d(num_filters[0], num_filters[0], kernel_size=4, stride=2, padding=1),
                                   nn.BatchNorm1d(num_filters[0]),
                                   nn.ReLU(),
                                   nn.MaxPool1d(kernel_size=(3,), stride=2,),
                                   #*residual_blocks,
                                   GAP(),
                                   nn.ReLU(),
                                   nn.Linear(num_filters[0], target_size),
                                   )
        
        
    def forward(self, x):
        '''
            x : size = (batch, input_size, maxlen)
        '''
        return self.model(x)        
        
        

net = Net(input_size=39,
          target_size=len(dataset['train'].targets_frame.columns)-2,
          num_layers = [1,1,1], num_filters = [128,128,128])
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
else:
    print("Single GPU mode")

net.to(device)

# Define criterion

In [ ]:
# define criterion
criterion = nn.MSELoss()
# criterion = nn.L1Loss()

def Normal(x, mu=0.0, sigma=1.0):
    if type(x)==torch.Tensor:
        x = x.detach().cpu().numpy()
        
    res = (1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2) )).astype(np.float32)
    return torch.tensor(res).float().to(device)


df_list = []  # list of df to merge

for phase in ['train', 'test']:
    df_list.append(
        dataset[phase].targets_frame.iloc[:,:-2]
    )

df = pd.concat(df_list)  #  concat

# mean and std from data
mu = df.values.mean(0)
sigma = df.values.std(0)

# criterion = lambda x, y : torch.mean( torch.pow( (x - y), 2) )
# criterion = lambda x, y : torch.mean( torch.pow(1-Normal(x, mu, sigma), 2) * torch.pow( (x - y), 2) )
# criterion = lambda x, y : torch.mean( torch.abs(torch.log( 1. / (Normal(x, mu, sigma)+1e-7) )) * torch.pow( (x - y), 2) )
# criterion = lambda x, y : torch.mean( 
#     torch.tanh( torch.abs(torch.log( 1. / (Normal(x, mu, sigma)) )) ) * torch.pow( (x - y), 2) )

# Training Loop

In [ ]:
# cost history saving..
history = {'train': [],
           'test': []}

import torch.optim as optim
from torch.optim import lr_scheduler

# Observe that all parameters are being optimized
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
# optimizer = optim.RMSprop(net.parameters(), lr=1e-2)
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, nesterov=True)

epoch_loss = {'train': 0.0, 'test': 0.0}
pred_and_gt = { k:[] for k in df.columns.values }

end_epoch = 20

print("Start training loop...")

for epoch in range(1,end_epoch+1):        
    for phase in ['train', 'test']:
        if phase=='train':
            net.train()
        elif phase=='test':
            net.eval()
        
        running_loss = 0.0

        for idx, batch_item in enumerate(dataloader[phase]):
            input, target, seq_len = batch_item['keypoints_seq'].to(device), batch_item['targets'].to(device), \
                                      batch_item['seq_len'].to(device)
                        
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=='train'):
                # feed data to network
                res = net(input)
                
                # compute loss
                loss = criterion(res, target)
                print(loss.item())
                if phase=='train':
                    loss.backward()
                    optimizer.step()
                                    
                if epoch==end_epoch and phase=='test':
                    for i,col in enumerate(df.columns.values):
                        pred_and_gt[col].append([res[:,i].detach().cpu().numpy(), target[:,i].cpu().numpy()])
                        
            running_loss += loss.item() * len(input)
            
            
        avg_loss = running_loss / len(dataloader[phase].dataset)
                
        epoch_loss[phase] += avg_loss
        
        if epoch % 5 == 0:
            print('=================={}========================'.format(phase.upper()))
            print('EPOCH : {}, WEIGHTED_SQUARED_ERROR (WSE) : {:.4f}'.format(epoch, epoch_loss[phase] / 5))
            history[phase].append(epoch_loss[phase] / 5)
            
            # init epoch_loss at its own phase
            epoch_loss[phase] = 0.0

print("Done!!")

In [ ]:
# plot learning curve
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.plot(np.arange(len(history['train'])*10, step=10), history['train'], label='train', color='b')
ax.plot(np.arange(len(history['test'])*10, step=10), history['test'], label='test', color='orange')

ax.set_title('Learning Curve')
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE')
plt.legend()

print("Saving learning curve...")
fig.savefig('learning.png', dpi=fig.dpi)

In [ ]:
import collections

scaler = dataset['train'].target_scaler
data = collections.defaultdict(list)

pp = []
gg = []
for i,col in enumerate(pred_and_gt.keys()):
    transposed_data = list(zip(*pred_and_gt[col]))
    preds = scaler.inverse_transform(np.concatenate(transposed_data[0]))
    gts = scaler.inverse_transform(np.concatenate(transposed_data[1]))
    
    pp.append(preds)
    gg.append(gts)
    
    for p,g in zip(preds, gts):
        data["name"].append(col)
        data["pred"].append(p)
        data["gt"].append(g)
        
df = pd.DataFrame(data)

grid = sns.FacetGrid(df, col="name", col_wrap=3, height=5, aspect=1.5, sharex=False, sharey=False)
grid.map(plt.scatter, "pred", "gt", color='g', label='data')

for i,(preds,gts) in enumerate(zip(pp,gg)):
    grid.axes[i].plot([min(preds), max(preds)], [min(gts), max(gts)], 'r--', label='GT=PRED')
    grid.axes[i].legend()